7. Pessoas que recebem certos tipos de notificações fazem mais churn

In [ ]:
from google.colab import auth
import pandas as pd

# get credentials
auth.authenticate_user()
query = '''
WITH notifications_churner AS (SELECT t.reason, t.user_id,u.churner
FROM `e-tensor-411113.neobankclean.notifications_clean` t
LEFT JOIN `e-tensor-411113.neobankclean.userdevice_clean` u USING(user_id))

SELECT reason,
COUNT(reason) AS total_notifications,
COUNT(DISTINCT(user_id)) AS users_per_notification,
ROUND(COUNT(DISTINCT(user_id))/COUNT(reason),2) AS pct_users_notified,
ROUND(COUNT(reason)/COUNT(DISTINCT(user_id)),2) AS notifications_per_user,
ROUND(SUM(churner)/COUNT(churner),2) AS churner_rate

FROM notifications_churner
GROUP BY reason

ORDER BY churner_rate DESC
'''

query_nao_agregada = '''
WITH notifications_churner AS (SELECT t.reason, t.user_id,u.churner
FROM `e-tensor-411113.neobankclean.notifications_clean` t
LEFT JOIN `e-tensor-411113.neobankclean.userdevice_clean` u USING(user_id))

SELECT reason,
user_id,
churner
FROM notifications_churner
'''

df = pd.read_gbq(query, project_id="neural-tangent-411113")
#df_n_agreg = pd.read_gbq(query_nao_agregada, project_id="neural-tangent-411113")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   reason                  17 non-null     object 
 1   total_notifications     17 non-null     Int64  
 2   users_per_notification  17 non-null     Int64  
 3   pct_users_notified      17 non-null     float64
 4   notifications_per_user  17 non-null     float64
 5   churner_rate            17 non-null     float64
dtypes: Int64(2), float64(3), object(1)
memory usage: 978.0+ bytes


In [ ]:
df

,reason,total_notifications,users_per_notification,pct_users_notified,notifications_per_user,churner_rate
0,JOINING_ANNIVERSARY,4158,4158,1.00,1.00,0.21
1,NO_INITIAL_CARD_ORDER,11136,7313,0.66,1.52,0.19
2,NO_INITIAL_FREE_PROMOPAGE_CARD_ORDER,5459,2723,0.50,2.00,0.15
3,REENGAGEMENT_ACTIVE_FUNDS,35844,12494,0.35,2.87,0.14
4,NO_INITIAL_CARD_USE,11221,4390,0.39,2.56,0.14
5,METAL_GAME_START,8373,8052,0.96,1.04,0.13
6,FIFTH_PAYMENT_PROMO,23743,11995,0.51,1.98,0.12
7,METAL_RESERVE_PLAN,1233,198,0.16,6.23,0.09
8,ONBOARDING_TIPS_ACTIVATED_USERS,3145,791,0.25,3.98,0.09
9,PREMIUM_ENGAGEMENT_INACTIVE_CARD,1124,595,0.53,1.89,0.06


In [ ]:
round(df.describe(),3)

,total_notifications,users_per_notification,pct_users_notified,notifications_per_user,churner_rate
count,17.0,17.0,17.000,17.000,17.000
mean,7165.471,3714.412,0.603,2.091,0.084
std,9411.097,3953.985,0.250,1.312,0.069
min,678.0,198.0,0.160,1.000,0.000
25%,1233.0,779.0,0.500,1.280,0.020
50%,3506.0,2054.0,0.530,1.890,0.090
75%,8373.0,4390.0,0.780,2.000,0.140
max,35844.0,12494.0,1.000,6.230,0.210


In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Criando a figura
fig = go.Figure()

# Adicionando o gráfico de barras
fig.add_trace(
    go.Bar(x=df['reason'], y=df['total_notifications'], name='Notificações')
             )

fig.add_trace(
    go.Bar(x=df['reason'], y=df['users_per_notification'], name='Usuários notificados')
             )

# Adicionando o gráfico de linhas
fig.add_trace(
    go.Scatter(x=df['reason'], y=df['churner_rate'], mode='lines+markers', name='Taxa de churn', yaxis='y2')
             )

fig.add_trace(
    go.Scatter(x=df['reason'], y=df['pct_users_notified'], mode='lines+markers', name='Taxa de usuários notificados', yaxis='y2')
             )

# Atualizando o layout para adicionar um segundo eixo y
fig.update_layout(
    title='Volume de notificações, usuários notificados, taxas de notificação e churn por tipo de notificação',
    xaxis=dict(tickfont=dict(size=8)),
    yaxis=dict(title='Valores absolutos'),
    yaxis2=dict(title='Percentagens', overlaying='y', side='right')
                 )

# Exibindo o gráfico
fig.show()

HO: Não há relação entre o tipo de transação e o churn rate

H1: Há relação entre o tipo de transação e o churn rate

**CONCLUSÃO:** há variação na taxa de churn, porém ela é pouco significativa, sugerindo a rejeição da hipótese alternativa.

numero de notificações por usuário

In [ ]:
# Criando a figura
fig = go.Figure()

# Adicionando o gráfico de barras
fig.add_trace(
    go.Bar(x=df['reason'], y=df['notifications_per_user'], name='Notificações')
             )

#fig.add_trace(go.Scatter(x=df['reason'], y=df['churner_rate'], mode='lines+markers', name='Taxa de churn', yaxis='y2'))

fig.show()

# Agrupamento de tipos de notificações

## Boas-vindas
- JOINING_ANNIVERSARY: boas-vindas como novo cliente
- ONBOARDING_TIPS_ACTIVATED_USERS: dicas de bondas-vindas
- WELCOME_HOME: boas-vindas após longo período de inatividade

## Promoções/benefícios
- BLACK_FRIDAY: promoção de Black Friday
- FIFTH_PAYMENT_PROMO: promoção concedida ao realizar a 5ª transferência
- PUMPKIN_PAYMENT_NOTIFICATION: pagamento com uso do emoji 🎃

`(in Halloween, if you send money to another user and you add a pumpkin emoji (🎃) into the notes of the transaction, you can win a Metal Card fee for a year.)`
- PREMIUM_ENGAGEMENT_FEES_SAVED: taxas economizadas do Premium user

## Alertas
- LOST_CARD_ORDER: pedido de cartão perdido
- MADE_MONEY_REQUEST_NOT_SPLIT_BILL: pedido pago em dinheiro, conta não dividida
- ENGAGEMENT_SPLIT_BILL_RESTAURANT: pagamento do pedido dividido
- METAL_RESERVE_PLAN: plano Metal adquirido
- PREMIUM_ENGAGEMENT_INACTIVE_CARD: cartão inativo do Premium user
- NO_INITIAL_FREE_PROMOPAGE_CARD_ORDER:
- NO_INITIAL_CARD_ORDER:
- REENGAGEMENT_ACTIVE_FUNDS:
- NO_INITIAL_CARD_USE:
- METAL_GAME_START:


In [ ]:
string = 'FEITO PEDIDO DE DINHEIRO, NÃO DIVIDIDO CONTA'
print(string.lower())

feito pedido de dinheiro, não dividido conta


In [ ]:
dicionario = {
              'JOINING_ANNIVERSARY':'Greeting alerts',
              'ONBOARDING_TIPS_ACTIVATED_USERS':'Greeting alerts',
              'WELCOME_HOME':'Greeting alerts',
              'BLACK_FRIDAY':'Promotion alerts',
              'FIFTH_PAYMENT_PROMO':'Promotion alerts',
              'PUMPKIN_PAYMENT_NOTIFICATION':'Promotion alerts',
              'PREMIUM_ENGAGEMENT_FEES_SAVED':'Promotion alerts',
              'LOST_CARD_ORDER':'General alerts',
              'MADE_MONEY_REQUEST_NOT_SPLIT_BILL':'General alerts',
              'ENGAGEMENT_SPLIT_BILL_RESTAURANT':'General alerts',
              'METAL_RESERVE_PLAN':'General alerts',
              'PREMIUM_ENGAGEMENT_INACTIVE_CARD':'General alerts',
              'NO_INITIAL_FREE_PROMOPAGE_CARD_ORDER':'General alerts',
              'NO_INITIAL_CARD_ORDER':'General alerts',
              'REENGAGEMENT_ACTIVE_FUNDS':'General alerts',
              'NO_INITIAL_CARD_USE':'General alerts',
              'METAL_GAME_START':'General alerts'
             }

#df['reason_type'] = dicionario[df['reason']]
df['reason_type'] = df['reason'].map(dicionario)

In [ ]:
df

,reason,total_notifications,users_per_notification,pct_users_notified,notifications_per_user,churner_rate,reason_type
0,JOINING_ANNIVERSARY,3873,3873,1.00,1.00,0.21,Greeting alerts
1,NO_INITIAL_CARD_ORDER,10158,6597,0.65,1.54,0.19,General alerts
2,NO_INITIAL_FREE_PROMOPAGE_CARD_ORDER,4932,2444,0.50,2.02,0.15,General alerts
3,NO_INITIAL_CARD_USE,10142,3874,0.38,2.62,0.14,General alerts
4,REENGAGEMENT_ACTIVE_FUNDS,32719,11162,0.34,2.93,0.14,General alerts
5,METAL_GAME_START,7583,7291,0.96,1.04,0.13,General alerts
6,FIFTH_PAYMENT_PROMO,21220,10700,0.50,1.98,0.12,Promotion alerts
7,METAL_RESERVE_PLAN,1096,175,0.16,6.26,0.09,General alerts
8,ONBOARDING_TIPS_ACTIVATED_USERS,2792,700,0.25,3.99,0.09,Greeting alerts
9,PREMIUM_ENGAGEMENT_INACTIVE_CARD,1040,548,0.53,1.90,0.06,General alerts


In [ ]:
#import plotly.express as px

#fig = px.bar(df, x='reason_type', y='notifications_per_user')

pivot = pd.pivot_table(df,
                       index=['reason_type'], values=['total_notifications','users_per_notification'], aggfunc='sum',
                       dropna=True)

pivot['pct_users_notified'] = round(pivot['users_per_notification']/pivot['total_notifications'],2)
pivot['notifications_per_user'] = round(pivot['total_notifications']/pivot['users_per_notification'],2)

In [ ]:
fig = px.bar(pivot, x=pivot.index, y='notifications_per_user')
fig.show()

In [ ]:
pivot

,total_notifications,users_per_notification,pct_users_notified,notifications_per_user
reason_type,,,,
General alerts,77019,37487,0.49,2.05
Greeting alerts,7467,5229,0.7,1.43
Promotion alerts,24203,13254,0.55,1.83


# Classification

In [ ]:
auth.authenticate_user()
query = '''
WITH transactions AS (
SELECT

--reason,
--channel,
--status,
n.user_id AS user_id,
--n.created_date,
u.churner,
CASE
  WHEN reason IN ('JOINING_ANNIVERSARY','ONBOARDING_TIPS_ACTIVATED_USERS','WELCOME_HOME') THEN 'Greeting alerts'
  WHEN reason IN ('BLACK_FRIDAY','FIFTH_PAYMENT_PROMO','PUMPKIN_PAYMENT_NOTIFICATION','PREMIUM_ENGAGEMENT_FEES_SAVED') THEN 'Promotion alerts'
  WHEN reason IN ('LOST_CARD_ORDER','MADE_MONEY_REQUEST_NOT_SPLIT_BILL','ENGAGEMENT_SPLIT_BILL_RESTAURANT','METAL_RESERVE_PLAN',
                  'PREMIUM_ENGAGEMENT_INACTIVE_CARD','NO_INITIAL_FREE_PROMOPAGE_CARD_ORDER','NO_INITIAL_CARD_ORDER',
                  'REENGAGEMENT_ACTIVE_FUNDS','NO_INITIAL_CARD_USE','METAL_GAME_START') THEN 'General alerts'
  ELSE null
END AS reason_type

FROM `e-tensor-411113.neobankclean.notifications_clean` n
LEFT JOIN `e-tensor-411113.neobankclean.userdevice_clean` u USING(user_id)
)

SELECT
user_id,
CASE WHEN reason_type IN ('Promotion alerts') THEN 1 ELSE 0 END AS Promotion_alerts,
CASE WHEN reason_type IN ('Greeting alerts') THEN 1 ELSE 0 END AS Greeting_alerts,
CASE WHEN reason_type IN ('General alerts') THEN 1 ELSE 0 END AS General_alerts
FROM transactions
--GROUP BY user_id
'''

df = pd.read_gbq(query, project_id="neural-tangent-411113")

In [ ]:
df

,user_id,churner,reason_type
0,user_7573,0,Promotion alerts
1,user_2750,0,Promotion alerts
2,user_5619,0,Promotion alerts
3,user_5197,0,Promotion alerts
4,user_1281,0,Promotion alerts
...,...,...,...
108684,user_18016,0,General alerts
108685,user_16284,0,General alerts
108686,user_10560,0,General alerts
108687,user_11328,0,General alerts


In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_train_ohe = enc.fit_transform(X_train)
X_test_ohe = enc.transform(X_test)

In [ ]:
from sklearn.preprocessing import LabelEncoder

y_not_encoded = df.pop('reason')

# Encoding label
label_enc = LabelEncoder()
df['reason'] = label_enc.fit_transform(y_not_encoded)
df.head()

,channel,status,user_id,churner,reason_type,reason
0,PUSH,SENT,user_7573,0,Promotion alerts,0
1,PUSH,SENT,user_2750,0,Promotion alerts,0
2,PUSH,SENT,user_5619,0,Promotion alerts,0
3,PUSH,SENT,user_5197,0,Promotion alerts,0
4,PUSH,SENT,user_1281,0,Promotion alerts,0


In [ ]:
from sklearn.model_selection import train_test_split

# every columns in X variable except churner which is our target
X = df.drop(columns=['churner'])
y = df['churner']

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# apply same transformation on X_test
X_test = scaler.transform(X_test)

In [ ]:
type(X_train)
type(X_test)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

baseline_y_pred = pd.Series(np.ones([95350]))

baseline_accuracy = accuracy_score(y_test, baseline_y_pred)

print(f"Baseline accuracy is {round(baseline_accuracy,2)}")

In [ ]:
from sklearn.linear_model import LogisticRegression

# train model
clf = LogisticRegression()
clf.fit(X_train, y_train)

# store predictions
y_pred = clf.predict(X_test)